## Week 8 Class 2: IP - Facililty Location Example 7.3

# Table of Contents<a id="Top"></a>

1. [Problem Statement](#1)<br>
2. [Data](#2) <br>
3. [Original Model Definition](#3)<br>
4. [Original Model Solution](#4)<br>
5. [Additional Constraints (Logical/Disjunctive) Model Definition](#5)<br>
6. [Additional Constraints (Logical/Disjunctive) Model Solution](#6)<br>

## 1. Problem Statement<a id=1></a>

A Facility Location problem is a blend of a transportation model with fixed costs. That is, we have a problem of moving products from up to four cities (New York, Atlanta, Chicago and Los Angeles) to the distribution centers in the East, South, Midwest and West.

##### [Back to Top](#Top)

## 2. Data<a id=2></a>

In [ ]:
import pandas as pd
import pyomo.environ as pe

You will have to change the path (below) to point to the file on your computer!

In [ ]:
raw_data = pd.read_excel('../Class-One/w08-c01-logical-part-2.xlsx', 
                         sheet_name = 'Example 7.3 - Location')
raw_data

In [ ]:
DV_indexes = ['E', 'S', 'M', 'W']
coef = pd.DataFrame(raw_data.iloc[range(2, 6), range(2, 6)])
coef.index = ['N', 'A', 'C','L']
coef.columns = DV_indexes
coef

In [ ]:
capcost = raw_data.iloc[range(2, 6), [6, 7]]
capcost.columns = ['capacity', 'annualcost']
capcost.index = coef.index
capcost

In [ ]:
demand = pd.DataFrame()
demand.loc['E', 'demand'] = 100
demand.loc['S', 'demand'] = 150
demand.loc['M', 'demand'] = 110
demand.loc['W', 'demand'] = 90
demand

##### [Back to Top](#Top)

### 3. Original Model Definition<a id=3></a>

In [ ]:
model = pe.ConcreteModel()

## Define Decision Variables

Define your decision variables. __NOTE:__ In this first problem, we have two sets of decision variables in our model. The `x` variables defined the quantities being shipped between each potential origin and the destination (E, M, S, and W). The `y` variables are the individual binary variables that define whether or not we use a particular facility. The variable `y` is indexed by the facility cities.

In [ ]:
model.N = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.A = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.C = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.L = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.y = pe.Var(coef.index, domain = pe.Binary)

for DV in model.component_objects(pe.Var):
    DV.pprint()

### Define Objective Function

Define your model objective function. Note that this will be the total costs which is calculated from the fixed costs and variable sales costs. This objective function represents but `sumproduct` function calls in the Excel worksheet. 

In [ ]:
[coef.loc['N', index]*model.N[index] for index in DV_indexes]

In [ ]:
#obj funct minimize cost * (N A C L) + annual cost * y
model.obj = pe.Objective(expr = sum(coef.loc['N', index]*model.N[index] for index in DV_indexes)
                                    + sum(coef.loc['A', index]*model.A[index] for index in DV_indexes)
                                    + sum(coef.loc['C', index]*model.C[index] for index in DV_indexes)
                                    + sum(coef.loc['L', index]*model.L[index] for index in DV_indexes)
                                    + sum(capcost.loc[index, 'annualcost']*model.y[index] 
                                          for index in coef.index),
                         sense = pe.minimize)
model.obj.pprint()

### Define constraints

In [ ]:
demand.loc['E', 'demand']

In [ ]:
#Demand Constraints
model.constE = pe.Constraint(expr = model.N['E'] + model.A['E'] + model.C['E'] + model.L['E'] 
                             >= demand.loc['E', 'demand'])
model.constS = pe.Constraint(expr = model.N['S'] + model.A['S'] + model.C['S'] + model.L['S']
                             >= demand.loc['S', 'demand'])
model.constM = pe.Constraint(expr = model.N['M'] + model.A['M'] + model.C['M'] + model.L['M']
                             >= demand.loc['M', 'demand'])
model.constW = pe.Constraint(expr = model.N['W'] + model.A['W'] + model.C['W'] + model.L['W']
                             >= demand.loc['W', 'demand'])

# Capacity and Linking Constraints DV <= M*y
model.constlinkN = pe.Constraint(expr=sum(model.N[index] for index in DV_indexes) 
                                 <= capcost.loc['N', 'capacity'] * model.y['N'])
model.constlinkA = pe.Constraint(expr=sum(model.A[index] for index in DV_indexes) 
                                 <= capcost.loc['A', 'capacity']* model.y['A'])
model.constlinkC = pe.Constraint(expr=sum(model.C[index] for index in DV_indexes) 
                                 <= capcost.loc['C', 'capacity'] * model.y['C'])
model.constlinkL = pe.Constraint(expr=sum(model.L[index] for index in DV_indexes) 
                                 <= capcost.loc['L', 'capacity'] * model.y['L'])

for con in model.component_objects(pe.Constraint):
    print(con,con.pprint())

##### [Back to Top](#Top)

### 4. Original Model Solution<a id=4></a>

One thing that I always fail to mention is that you absolutely do not need to redefine the `opt` object each time you run `opt.solve` on a model. The `opt` definition will take care of specifying to use `glpk` throughout the entire python session.

In [ ]:
opt = pe.SolverFactory('glpk')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

### Optimal Objective Value

In [ ]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

### Optimal Decision Variables

In [ ]:
DV_solution = pd.DataFrame()
for index in DV_indexes:
    DV_solution.loc['N', index] = model.N[index].value
    DV_solution.loc['A', index] = model.A[index].value
    DV_solution.loc['C', index] = model.C[index].value
    DV_solution.loc['L', index] = model.L[index].value
DV_solution

The following shows which facilities are being utilized.

In [ ]:
for index in coef.index:
    print(f'y{index}:', model.y[index].value)

##### [Back to Top](#Top)

### 5. Additional Constraints (Logical/Disjunctive) Model Definition<a id=5></a>

In each subsequent problem, we will just add constraints to the problem. We don't need to redefine the model, objective function, etc. if they aren't changing (and they aren't here). We are just adding constraints to the problems and this will **might** require adding new variables and we have to in this case.

Let's look at the coefficients associated with our objective functions and constraint equations again. 

In [ ]:
coef

In [ ]:
capcost

In [ ]:
demand

### 5.2 Threshold Problem

#### Define Decision Variables

In [ ]:
#BINARY Threshold NM > 60 or 0 Indicator
model.yNM60 = pe.Var(domain = pe.Binary)

In [ ]:
#New constraints
big_M = 150
#Threshold NM >=60 or must be 0: NM >= 60 * yNM and NM <= 150 * yNM
model.constNM601 = pe.Constraint(expr = model.N['M'] >= 60 * model.yNM60)
model.constNM602 = pe.Constraint(expr = model.N['M'] <= 150 * model.yNM60)

In [ ]:
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

In [ ]:
DV_solution = pd.DataFrame()
for index in DV_indexes:
    DV_solution.loc['N', index] = model.N[index].value
    DV_solution.loc['A', index] = model.A[index].value
    DV_solution.loc['C', index] = model.C[index].value
    DV_solution.loc['L', index] = model.L[index].value
DV_solution

In [ ]:
for index in coef.index:
    print(f'y{index}:', model.y[index].value)

In [ ]:
print(f'yNM60:', model.yNM60.value)

#### Define Decision Variables

Now we already have the DV for each path quantitiy `model.x` and the Fixed Cost binary indicator `y`. We need to add two more variables to the model. 

In [ ]:
#BINARY Constraint LE >= 50
model.yLE50 = pe.Var(domain = pe.Binary)
#BINARY Constraint NE <= 75
model.yNE75 = pe.Var(domain = pe.Binary)

Now define the additional constraints.

In [ ]:
#Disjunctive LE >= 50: LE−𝑀*yLE≤50 and LE+𝑀(1−𝑦LE)≥50

model.constLE501 = pe.Constraint(expr = model.L['E'] - big_M*model.yLE50 <= 50)
model.constLE502 = pe.Constraint(expr = model.L['E'] + big_M*(1 - model.yLE50) >= 50)

#Disjunctive NE <=75: NE+𝑀*yNE≥75 and NE−𝑀(1−𝑦NE)≤75
model.constNE751 = pe.Constraint(expr = model.N['E'] + big_M*model.yNE75 >= 75)
model.constNE752 = pe.Constraint(expr = model.N['E'] - big_M*(1 - model.yNE75) <= 75)

#Disjunctive Linking yLE + yNE >= 1
model.constLink = pe.Constraint(expr = model.yLE50 + model.yNE75 >= 1)

In [ ]:
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

In [ ]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

In [ ]:
DV_solution = pd.DataFrame()
for index in DV_indexes:
    DV_solution.loc['N', index] = model.N[index].value
    DV_solution.loc['A', index] = model.A[index].value
    DV_solution.loc['C', index] = model.C[index].value
    DV_solution.loc['L', index] = model.L[index].value
DV_solution

In [ ]:
for index in coef.index:
    print(f'y{index}:', model.y[index].value)

We should check that the linking variables seem to be working.
* The `y` variables check usage for each set of City paths. Here we see `y` is 0 because we did not use the Chicago paths.
* The `yLE50` checks if LE >= 50. Above we see LE final count is 0 which means it was not met which matches the y of 0
* the `yNE75` checks if NE <= 75. Above we see NE final count is 75 which means it did meet which matches the y of 1.

In [ ]:
print(f'yLE50:', model.yLE50.value)
print(f'yNE75:', model.yNE75.value)

In [ ]:
for index in coef.index:
    print(f'y{index}:', model.y[index].value)

In [ ]:
for con in model.component_objects(pe.Constraint):
    print(con.lower, con.slack(), con.upper)